In [1]:
import os
from rank_bm25 import BM25Okapi
import numpy as np
import csv
from tqdm import tqdm
from konlpy.tag import Mecab

In [2]:
mecab = Mecab("C:\mecab\mecab-ko-dic")
def mecab_tokenizer(sent):
    return mecab.morphs(sent)

In [3]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
file_path = os.path.join(parent_directory, 'aihub')

document

In [4]:
doc_id = []
corpus = []

with open(os.path.join(file_path, 'collection.tsv'), 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split('||', 1)
        docid = parts[0].strip()
        corpus_ = parts[1].strip()
        doc_id.append(docid)
        corpus.append(corpus_)

In [7]:
len(doc_id), len(corpus)

(124535, 124535)

tokenizer

In [8]:
tokenized_corpus = [mecab_tokenizer(doc) for doc in tqdm(corpus)]

  0%|          | 0/124535 [00:00<?, ?it/s]

100%|██████████| 124535/124535 [05:57<00:00, 348.38it/s] 


In [9]:
k1 = 1.2
b = 0.75
bm25 = BM25Okapi(tokenized_corpus, k1=k1, b = b)

쿼리

In [11]:
questions = {}

with open(os.path.join(file_path, 'questions.tsv'), 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split('\t', 1)
        qid = parts[0].strip()
        question = parts[1].strip()
        if qid in questions.keys():
            print('1')
        else:
            questions[qid] = question


In [12]:
len(questions)

252689

정답

In [13]:
test_qids = []
test_docid = []

with open(os.path.join(file_path, 'test/qrels_test.tsv'), 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split('\t', 1)
        qid = parts[0].strip()
        docid = parts[1].strip()
        test_qids.append(qid)
        test_docid.append(docid)

In [14]:
len(test_qids), len(test_docid)

(3000, 3000)

test

In [15]:
recall1 = 0
recall2 = 0
recall5 = 0
recall10 = 0
recall20 = 0
recall100 = 0
recall200 = 0


for idx, test_qid in tqdm(enumerate(test_qids), total = len(test_qids)):
    query = questions[test_qid]
    answer = test_docid[idx]

    tokenized_query = mecab_tokenizer(query)
    doc_scores = bm25.get_scores(tokenized_query)
    
    top_indices = np.argsort(doc_scores)[-100:][::-1]

    for i, idx in enumerate(top_indices):
        if i < 1 and answer == doc_id[idx]:
            recall1 += 1
            recall2 += 1
            recall5 += 1
            recall10 += 1
            recall20 += 1
            recall100+=1
            recall200+=1
            break
        elif i < 2 and answer == doc_id[idx]:
            recall2 += 1
            recall100+=1
            recall200+=1
            recall5 += 1
            recall10 += 1
            recall20 += 1
        elif i < 5 and answer == doc_id[idx]:
            recall5 += 1
            recall10 += 1
            recall20 += 1
            recall100+=1
            recall200+=1
            break
        elif i < 10 and answer == doc_id[idx]:
            recall10 += 1
            recall20 += 1
            recall100+=1
            recall200+=1
            break
        elif i < 20 and answer == doc_id[idx]:
            recall20 += 1
            recall100+=1
            recall200+=1
            break
        elif i < 100 and answer == doc_id[idx]:
            recall100+=1
            recall200+=1
            break
        elif i < 200 and answer == doc_id[idx]:
            recall200+=1
            break

  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [1:53:11<00:00,  2.26s/it]  


In [16]:
print(recall1 / len(test_qids))
print(recall2 / len(test_qids))
print(recall5 / len(test_qids))
print(recall10 / len(test_qids))
print(recall20 / len(test_qids))
print(recall100 / len(test_qids))
print(recall200 / len(test_qids))

0.6346666666666667
0.77
0.866
0.9033333333333333
0.9396666666666667
0.974
0.974
